In [2]:
from json_file_func import *
import pandas as pd
import numpy as np
from df_functions import *

In [3]:
import csv
import json

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
import xgboost as xgb
import random

In [16]:
input_folder = r"C:\Users\user\Desktop\scy-project\processed_data\scy"
output_file = r"C:\Users\user\Desktop\scy-project\tables\merged_scy.json"

In [13]:
merge_json_files(input_folder, output_file)

In [17]:

# Define the JSON file path
json_file_path = output_file

# Read the JSON data from the file
with open(json_file_path) as file:
    data = json.load(file)

# Extract the list of labels from the JSON data
labels = data["abstractedFeatures"]

# Extract the entity data
entities = data["entitiesData"]

# Define the output CSV file path
csv_file_path = r"C:\Users\user\Desktop\scy-project\tables\label_count_scy.csv"

# Write the data to the CSV file
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Id"] + labels)

    # Write the data rows
    for entity in entities:
        entity_name = entity["entityName"]
        label_counts = {label: 0 for label in labels}  # Create a new dictionary for each entity

        for record in entity["records"]:
            label = record["label"]
            label_counts[label] += 1

        counts = [label_counts[label] for label in labels]
        writer.writerow([entity_name] + counts)

# Read the CSV file to check for empty columns
with open(csv_file_path, mode="r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Find the empty columns (all values are zero)
empty_columns = [index for index, column in enumerate(zip(*rows[1:])) if all(value == '0' for value in column)]

# Remove the empty columns from the data
for index in empty_columns[::-1]:
    for row in rows:
        del row[index]

# Write the updated data to the CSV file
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print("CSV file created successfully!")



CSV file created successfully!


In [33]:
norm_df = pd.read_csv(r"C:\Users\user\Desktop\scy-project\tables\label_count_norm.csv")
scy_df = pd.read_csv(r"C:\Users\user\Desktop\scy-project\tables\label_count_scy.csv")
norm_df['healthy'] = 1
scy_df['healthy'] = 0
df = pd.concat([norm_df, scy_df], axis=0, ignore_index=True)

In [20]:
len(df)

84

In [34]:
df=df.drop(['Id'],axis =1)

In [35]:
df.head()

,Channel_1-Alpha-L0,Channel_1-Alpha-L1,Channel_1-Alpha-L2,Channel_1-Beta-L0,Channel_1-Beta-L1,Channel_1-Beta-L2,Channel_1-Gamma-L0,Channel_1-Gamma-L1,Channel_1-Gamma-L2,Channel_2-Alpha-L0,...,Channel_16-Alpha-L0,Channel_16-Alpha-L1,Channel_16-Alpha-L2,Channel_16-Beta-L0,Channel_16-Beta-L1,Channel_16-Beta-L2,Channel_16-Gamma-L0,Channel_16-Gamma-L1,Channel_16-Gamma-L2,healthy
0,168,90,42,174,87,39,166,95,39,164,...,224,72,4,253,42,5,229,63,8,1
1,217,63,20,219,59,22,208,65,27,219,...,273,27,0,258,39,3,256,36,8,1
2,132,94,74,82,120,98,129,96,75,168,...,225,67,8,155,100,45,174,94,32,1
3,91,87,122,108,104,88,134,107,59,84,...,37,64,199,28,84,188,57,103,140,1
4,154,88,58,175,93,32,158,101,41,137,...,130,105,65,157,94,49,151,106,43,1


In [44]:
df.to_csv(r'C:\Users\user\Desktop\scy-project\tables\final_df.csv', index=False)

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define your function to train and predict using Random Forest
def train_random_forest(X_train, y_train, X_test, params):
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    return (y_pred >= 0.5).astype(int)  # Convert probabilities to binary predictions

# Define the hyperparameters for different Random Forest models
random_forest_params =  [
    {'n_estimators': 100, 'max_depth': 3},
    {'n_estimators': 100, 'max_depth': 4},
    {'n_estimators': 100, 'max_depth': 5},
    {'n_estimators': 100, 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1},
    {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 1},
    {'n_estimators': 130, 'max_depth': 3},
    {'n_estimators': 130, 'max_depth': 4},
    {'n_estimators': 130, 'max_depth': 5},
    {'n_estimators': 130, 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1},
    {'n_estimators': 130, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 1},
    {'n_estimators': 70, 'max_depth': 3},
    {'n_estimators': 70, 'max_depth': 4},
    {'n_estimators': 70, 'max_depth': 5},
    {'n_estimators': 70, 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1},
    {'n_estimators': 70, 'max_depth': 5, 'min_samples_split': 4, 'min_samples_leaf': 1},
]

X = df.drop(columns=['healthy'])
y = df['healthy']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize scaler and apply it
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for i, params in enumerate(random_forest_params):
    model_name = f'RandomForest{i+1}'
    model = RandomForestClassifier(**params)
    model.fit(scaled_X_train, y_train)
    y_pred = model.predict(scaled_X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = recall_score(y_test, y_pred, pos_label=0)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0

    # Store results
    params_str = ', '.join([f"{param}={val}" for param, val in params.items()])
    results[model_name] = {
        'Hyperparameters': params_str,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    }

# Create a DataFrame to display the results
combined_results = pd.DataFrame.from_dict(results, orient='index')
print(combined_results)



                                                  Hyperparameters  Accuracy  \
RandomForest1                       n_estimators=100, max_depth=3  0.961538   
RandomForest2                       n_estimators=100, max_depth=4  0.923077   
RandomForest3                       n_estimators=100, max_depth=5  0.961538   
RandomForest4   n_estimators=100, max_depth=3, min_samples_spl...  0.961538   
RandomForest5   n_estimators=100, max_depth=5, min_samples_spl...  0.923077   
RandomForest6                       n_estimators=130, max_depth=3  0.961538   
RandomForest7                       n_estimators=130, max_depth=4  0.961538   
RandomForest8                       n_estimators=130, max_depth=5  0.961538   
RandomForest9   n_estimators=130, max_depth=3, min_samples_spl...  0.923077   
RandomForest10  n_estimators=130, max_depth=5, min_samples_spl...  0.961538   
RandomForest11                       n_estimators=70, max_depth=3  0.923077   
RandomForest12                       n_estimators=70

In [38]:
combined_results.index.name = 'Model'
combined_results

,Hyperparameters,Accuracy,Sensitivity,Specificity,PPV,NPV
Model,,,,,,
RandomForest1,"n_estimators=100, max_depth=3",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest2,"n_estimators=100, max_depth=4",0.923077,0.916667,0.928571,0.916667,0.928571
RandomForest3,"n_estimators=100, max_depth=5",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest4,"n_estimators=100, max_depth=3, min_samples_spl...",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest5,"n_estimators=100, max_depth=5, min_samples_spl...",0.923077,0.916667,0.928571,0.916667,0.928571
RandomForest6,"n_estimators=130, max_depth=3",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest7,"n_estimators=130, max_depth=4",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest8,"n_estimators=130, max_depth=5",0.961538,1.000000,0.928571,0.923077,1.000000
RandomForest9,"n_estimators=130, max_depth=3, min_samples_spl...",0.923077,0.916667,0.928571,0.916667,0.928571


In [53]:

# Define your function to train and predict using SVC
def train_svc(X_train, y_train, X_test, params):
    model = SVC(**params, probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    return (y_pred >= 0.5).astype(int)  # Convert probabilities to binary predictions

models = ['SVC' + str(i+1) for i in range(18)]
metrics = ['Accuracy', 'Sensitivity', 'Specificity', 'PPV', 'NPV']
predictions = {}

# Define the hyperparameters for different SVC models
svc_params =  [
    {'C': 0.4, 'kernel': 'rbf'},
    {'C': 0.5, 'kernel': 'rbf'},
    {'C': 0.6, 'kernel': 'rbf'},
    {'C': 0.7, 'kernel': 'rbf'},
    {'C': 0.8, 'kernel': 'rbf'},
    {'C': 0.9, 'kernel': 'rbf'},
    {'C': 1, 'kernel': 'rbf'},
    {'C': 1.1, 'kernel': 'rbf'},
    {'C': 1.2, 'kernel': 'rbf'},
    {'C': 1.3, 'kernel': 'rbf'},
    {'C': 1.4, 'kernel': 'rbf'},
    {'C': 1.5, 'kernel': 'rbf'},
        {'C': 1.6, 'kernel': 'rbf'},
    {'C': 1.7, 'kernel': 'rbf'},
    {'C': 1.8, 'kernel': 'rbf'},
    {'C': 1.9, 'kernel': 'rbf'},
    {'C': 2, 'kernel': 'rbf'},
    {'C': 2.1, 'kernel': 'rbf'}
]

X = df.drop(columns=['healthy'])
y = df['healthy']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize scaler and apply it
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for i, params in enumerate(svc_params):
    model_name = f'SVC{i+1}'
    model = SVC(**params)
    model.fit(scaled_X_train, y_train)
    y_pred = model.predict(scaled_X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = recall_score(y_test, y_pred, pos_label=0)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0

    # Store results
    params_str = ', '.join([f"{param}={val}" for param, val in params.items()])
    results[model_name] = {
        'Hyperparameters': params_str,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    }

# Create a DataFrame to display the results
combined_results = pd.DataFrame.from_dict(results, orient='index')
print(combined_results)



         Hyperparameters  Accuracy  Sensitivity  Specificity       PPV  \
SVC1   C=0.4, kernel=rbf  0.884615     0.750000     1.000000  1.000000   
SVC2   C=0.5, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC3   C=0.6, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC4   C=0.7, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC5   C=0.8, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC6   C=0.9, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC7     C=1, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC8   C=1.1, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC9   C=1.2, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC10  C=1.3, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC11  C=1.4, kernel=rbf  0.846154     0.833333     0.857143  0.833333   
SVC12  C=1.5, kernel=rbf  0.884615     0.916667     0.857143  0.846154   
SVC13  C=1.6, kernel=rbf  0.884615    

In [54]:
combined_results.index.name = 'Model'
combined_results

,Hyperparameters,Accuracy,Sensitivity,Specificity,PPV,NPV
Model,,,,,,
SVC1,"C=0.4, kernel=rbf",0.884615,0.750000,1.000000,1.000000,0.823529
SVC2,"C=0.5, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC3,"C=0.6, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC4,"C=0.7, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC5,"C=0.8, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC6,"C=0.9, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC7,"C=1, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC8,"C=1.1, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143
SVC9,"C=1.2, kernel=rbf",0.846154,0.833333,0.857143,0.833333,0.857143


In [49]:
# Define your function to train and predict using SVC
def train_svc(X_train, y_train, X_test, params):
    model = SVC(**params, probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    return (y_pred >= 0.5).astype(int)  # Convert probabilities to binary predictions

models = ['SVC' + str(i+1) for i in range(20)]
metrics = ['Accuracy', 'Sensitivity', 'Specificity', 'PPV', 'NPV']
predictions = {}

# Define the hyperparameters for different SVC models
svc_params =  [
    {'C': 0.005, 'kernel': 'poly'},
    {'C': 0.01, 'kernel': 'poly'},
    {'C': 0.02, 'kernel': 'poly'},
    {'C': 0.05, 'kernel': 'poly'},
    {'C': 0.1, 'kernel': 'poly'},
    {'C': 0.25, 'kernel': 'poly'},
    {'C': 0.5, 'kernel': 'poly'},
    {'C': 0.6, 'kernel': 'poly'},
    {'C': 0.7, 'kernel': 'poly'},
    {'C': 0.8, 'kernel': 'poly'},
    {'C': 0.9, 'kernel': 'poly'},
    {'C': 1.0, 'kernel': 'poly'},
    {'C': 1.1, 'kernel': 'poly'},
    {'C': 1.2, 'kernel': 'poly'},
    {'C': 1.3, 'kernel': 'poly'},
    {'C': 1.5, 'kernel': 'poly'},
    {'C': 1.7, 'kernel': 'poly'},
    {'C': 1.9, 'kernel': 'poly'},
    {'C': 3, 'kernel': 'poly'},
    {'C': 5, 'kernel': 'poly'}
]


X = df.drop(columns=['healthy'])
y = df['healthy']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize scaler and apply it
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for i, params in enumerate(svc_params):
    model_name = f'SVC{i+1}'
    model = SVC(**params)
    model.fit(scaled_X_train, y_train)
    y_pred = model.predict(scaled_X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = recall_score(y_test, y_pred, pos_label=0)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0

    # Store results
    params_str = ', '.join([f"{param}={val}" for param, val in params.items()])
    results[model_name] = {
        'Hyperparameters': params_str,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv
    }

# Create a DataFrame to display the results
combined_results = pd.DataFrame.from_dict(results, orient='index')
print(combined_results)



            Hyperparameters  Accuracy  Sensitivity  Specificity       PPV  \
SVC1   C=0.005, kernel=poly  0.538462     0.000000     1.000000  0.000000   
SVC2    C=0.01, kernel=poly  0.538462     0.000000     1.000000  0.000000   
SVC3    C=0.02, kernel=poly  0.538462     0.000000     1.000000  0.000000   
SVC4    C=0.05, kernel=poly  0.538462     0.000000     1.000000  0.000000   
SVC5     C=0.1, kernel=poly  0.538462     0.000000     1.000000  0.000000   
SVC6    C=0.25, kernel=poly  0.576923     0.166667     0.928571  0.666667   
SVC7     C=0.5, kernel=poly  0.653846     0.333333     0.928571  0.800000   
SVC8     C=0.6, kernel=poly  0.692308     0.416667     0.928571  0.833333   
SVC9     C=0.7, kernel=poly  0.769231     0.583333     0.928571  0.875000   
SVC10    C=0.8, kernel=poly  0.769231     0.583333     0.928571  0.875000   
SVC11    C=0.9, kernel=poly  0.730769     0.500000     0.928571  0.857143   
SVC12    C=1.0, kernel=poly  0.730769     0.500000     0.928571  0.857143   

In [50]:
combined_results.index.name = 'Model'
combined_results

,Hyperparameters,Accuracy,Sensitivity,Specificity,PPV,NPV
Model,,,,,,
SVC1,"C=0.005, kernel=poly",0.538462,0.000000,1.000000,0.000000,0.538462
SVC2,"C=0.01, kernel=poly",0.538462,0.000000,1.000000,0.000000,0.538462
SVC3,"C=0.02, kernel=poly",0.538462,0.000000,1.000000,0.000000,0.538462
SVC4,"C=0.05, kernel=poly",0.538462,0.000000,1.000000,0.000000,0.538462
SVC5,"C=0.1, kernel=poly",0.538462,0.000000,1.000000,0.000000,0.538462
SVC6,"C=0.25, kernel=poly",0.576923,0.166667,0.928571,0.666667,0.565217
SVC7,"C=0.5, kernel=poly",0.653846,0.333333,0.928571,0.800000,0.619048
SVC8,"C=0.6, kernel=poly",0.692308,0.416667,0.928571,0.833333,0.650000
SVC9,"C=0.7, kernel=poly",0.769231,0.583333,0.928571,0.875000,0.722222
